In [ ]:
# rag_ollama_chroma.py
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_community.llms import Ollama
from langchain_ollama import ChatOllama
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
from langchain_core.prompts import ChatPromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

import os
from pathlib import Path

In [ ]:
root_path = Path(os.getcwd()).parent.parent
ollama_model = root_path / "ollama-service" / "models" / "ko_llama" / "llama-3-Korean-Bllossom-8B.Q4_K_M.gguf"
embeddings_path = root_path / "ollama-service" / "models" / "BGE-m3-ko"
print(ollama_model)
print(embeddings_path)

In [ ]:
# HuggingFace 임베딩 모델 로드 (경로를 문자열로 변환하여 오류 방지)
embedding_model = HuggingFaceEmbeddings(
    model_name=str(embeddings_path)
)
# llm = Ollama(model=str(ollama_model))

In [ ]:
vectorstore = FAISS.load_local("vector_db/pcn_web", embedding_model, allow_dangerous_deserialization=True)

In [ ]:
query_embedding = embedding_model.embed_query("피씨엔 소개")

# 벡터 유사도 검색 (상위 3개 결과 반환)
results = vectorstore.similarity_search_by_vector(query_embedding, k=10)

In [ ]:
# 검색기(Retriever) 설정 (개선)
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})  # 상위 5개 문서 반환하도록 개선
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

llm = ChatOllama(
    model=str("ko-llama-8B"),
    temperature=0.1,  # 약간의 다양성 부여
    base_url="http://localhost:11434",
    callback_manager=callback_manager,
    retriever=retriever  # retriever를 llm에 직접 설정
)

# 프롬프트를 더 명확하고 자연스럽게 개선
system_template = (
    "당신은 친절하고 유능한 AI 어시스턴트입니다. "
    "사용자의 질문에 대해 신뢰할 수 있는 정보를 바탕으로 정확하고 간결하게 답변하세요."
)
human_template = (
    "아래의 질문에 대해 단계별로 논리적으로 생각하여 답변해 주세요.\n"
    "질문: {question}"
)

system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
chain = prompt | llm

In [ ]:
chain.invoke({"question": "피씨엔 소개"})

In [ ]:
# 3. 질의 응답 체인 구성
# 프롬프트 템플릿 정의
# context 변수에는 검색된 문서 내용이 들어갑니다.
# question 변수에는 사용자 질문이 들어갑니다.
template = """다음 맥락을 사용하여 질문에 답하세요:
{context}

질문: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

print("프롬프트 템플릿 준비 완료.")

# LangChain LCEL(LangChain Expression Language)을 사용하여 체인 구성
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print("RAG 체인 구성 완료. 이제 질문할 수 있습니다!")

# 4. 질의 응답 테스트
while True:
    user_question = input("\n질문하세요 (종료하려면 'q' 입력): ")
    if user_question.lower() == 'q':
        print("RAG 시스템을 종료합니다.")
        break

    print(f"'{user_question}' 질문에 대한 답변을 생성 중...")
    try:
        response = rag_chain.invoke(user_question)
        print("\n--- 답변 ---")
        print(response)
        print("------------")
    except Exception as e:
        print(f"오류 발생: {e}. Ollama 모델이 제대로 실행 중인지 확인하세요.")